# Evaluating a NeMo checkpoint on an arbitrary task

This notebook demonstrates how to extend the evaluation capabilities within the NeMo Framework container.
It guides you through the advanced configuration of an evaluation job.

For an introduction to evaluation with NeMo Evaluator and the NeMo Framework, please refer to the tutorial ["Evaluating a Megatron Bridge checkpoint with lm-eval"](mmlu.ipynb) first.

In this tutorial, we will evaluate an LLM on the [WikiText-2](https://arxiv.org/abs/1609.07843) benchmark, which is available in the [NeMo Evaluator lm-eval](https://pypi.org/project/nvidia-lm-eval/) package.
The evaluation utilizes the log-probabilities of the context tokens to assess how likely the input text is, according to the model.

> Note: It is recommended to run this notebook within a [NeMo Framework container](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/nemo), as it includes all necessary dependencies.

In [ ]:
import logging
import os
import signal
import subprocess

from nemo_evaluator.api import check_endpoint, evaluate, show_available_tasks
from nemo_evaluator.api.api_dataclasses import (
    ApiEndpoint,
    ConfigParams,
    EndpointType,
    EvaluationConfig,
    EvaluationTarget,
)

logging.getLogger().setLevel(logging.INFO)

## 1. Deploying the model

We will start from deploying the model.

First, you need to prepare a Megatron Bridge checkpoint of the model you would like to evaluate.

To evaluate a checkpoint saved during pre-training or fine-tuning with Megatron-Bridge, set the `CHECKPOINT_PATH` variable below to the path to the saved checkpoint. Make sure to mount the directory containing the checkpoint when starting the container.
Otherwise, Hugging Face checkpoints can be converted to Megatron Bridge.

For the purpose of this tutorial, we will pull the Llama 3.2 1B Instruct Hugging Face checkpoint and convert it into Megatron Bridge. Since Llama 3.2 1B Instruct is gated, make sure to authenticate with Hugging Face first.

Before proceeding to the evaluation, we will wait for the server to start accepting the requests using the `check_endpoint` function.

> Note: You can learn more about deployment and available server endpoints from the ["Evaluating a Megatron Bridge checkpoint with lm-eval"](mmlu.ipynb) tutorial.

In [ ]:
# if you have a Megatron Bridge checkpoint, modify this variable to point to its location
CHECKPOINT_PATH = "/workspace/llama-3_2-1b-instruct"
ITER = "iter_0000000"  # set to iternation sub-dir you want to use

# modify this path if you would like to save results in a different directory
WORKSPACE = "."

If you do not have a Magatron Bridge checkpoint ready, you can use the cell below to create one.

In [ ]:
# uncomment the lines below to authenticate with Hugging Face Hub
# os.environ["HF_TOKEN"] = "hf_..."  # set to your Hugging Face token

# create fresh checkpoint if not provided
if not os.path.exists(CHECKPOINT_PATH):
    from megatron.bridge import AutoBridge

    AutoBridge.import_ckpt("meta-llama/Llama-3.2-1B-Instruct", CHECKPOINT_PATH)

In [ ]:
host = "0.0.0.0"
port = "8886"
model_name = "megatron_model"
completions_url = f"http://{host}:{port}/v1/completions/"

In [ ]:
deploy_script = "/opt/Export-Deploy/scripts/deploy/nlp/deploy_ray_inframework.py"
# OOM with 1 GPU
deploy_process = subprocess.Popen(
    [
        "python",
        deploy_script,
        "--megatron_checkpoint",
        f"{CHECKPOINT_PATH}/{ITER}",
        "--model_id",
        model_name,
        "--host",
        host,
        "--port",
        port,
        "--tensor_model_parallel_size",
        "2",
        "--num_gpus",
        "2",
    ],
)

check_endpoint(
    endpoint_url=completions_url,
    endpoint_type=EndpointType.COMPLETIONS,
    model_name=model_name,
    max_retries=600,
)

## 2. Defining a custom evaluation workflow

NVIDIA Evals Factory packages include pre-defined evaluation configurations.
These configurations represent some of the most commonly used evaluation settings and simplify running the most frequently used benchmarks.

They can be listed using the `show_available_tasks` function.

In [ ]:
show_available_tasks()

However, users are not limited to this short list of benchmarks.
If you would like to evaluate a model on a different task from the underlying evaluation harness, you simply need to specify the full configuration.

For this tutorial, we will use the `wikitext` task from `lm-evaluation-harness`.
Note that for tasks without a predefined configuration you must specify the type in the `"<evaluation harness>.<task name>"` format.

Since this task uses the log-likelihoods of the input texts, we need to specify parameters for loading the tokenizer: `"tokenizer_backend"` and `"tokenizer"`.
For the model used in this example these are `"huggingface"` and `"/workspace/llama-3_2-1b-instruct/iter_0000000/tokenizer"`, respectively. You can also use a Hugging Face model handle directly for loading the tokenizer (`"meta-llama/Llama-3.2-1B-Instruct"`).

In [ ]:
target_config = EvaluationTarget(
    api_endpoint=ApiEndpoint(
        url=completions_url, type=EndpointType.COMPLETIONS, model_id=model_name
    )
)
eval_config = EvaluationConfig(
    type="lm-evaluation-harness.wikitext",
    params=ConfigParams(
        temperature=0,
        top_p=0,
        parallelism=1,
        extra={
            "tokenizer_backend": "huggingface",
            "tokenizer": f"{CHECKPOINT_PATH}/{ITER}/tokenizer",
        },
    ),
    output_dir=f"{WORKSPACE}/wikitext-results",
)
results = evaluate(target_cfg=target_config, eval_cfg=eval_config)

Finally, we can shut the model server down and inspect evaluation results.

In [ ]:
deploy_process.send_signal(signal.SIGKILL)

In [ ]:
print(results.tasks)